In [1]:
import numpy as np

In [56]:
class DecisionTreeLearner:
    def __init__(self, leaf_size = 1):
        self.LEAF = -1
        self.leaf_size = leaf_size
        self.tree = np.empty((1,1))

    def build_tree(self, data):
        if data.shape[0] <= self.leaf_size or np.all(data[:, -1] == data[:, -1][0]):
            return np.array([self.LEAF, np.mean(data[:,-1]), self.LEAF, self.LEAF], dtype=np.float64).reshape((1,4))
        else:
            i = self.find_split(data)
            SplitVal = np.median(data[:,i])
            if np.all(data[:,i]<=SplitVal):
                return np.array([self.LEAF, data[-1, 1], self.LEAF, self.LEAF], dtype=np.float64).reshape((1,4))
            elif np.all(data[:,i]>SplitVal):
                return np.array([self.LEAF, data[-1, 1], self.LEAF, self.LEAF], dtype=np.float64).reshape((1,4))
            lefttree = self.build_tree(data[data[:,i] <= SplitVal])
            righttree = self.build_tree(data[data[:,i] > SplitVal])
            root = np.array([i, SplitVal, 1, int(lefttree.shape[0]) + 1], dtype=np.float64).reshape((1,4))
            return np.concatenate((root, lefttree, righttree), axis=0)

    def find_split(self, data):
        coef_vals = np.zeros(data.shape[1]-1)
        for k in range(data.shape[1]-1):
            coef_vals[k] = np.abs(np.corrcoef(data[:,k], data[:,-1])[0,1])
        i = np.argmax(coef_vals)
        return i

    def train(self, data):
        self.tree = self.build_tree(data)

    def print_tree(self):
        for line in self.tree:
            print(line)

    def query(self, points):
        ans = np.array([])
        for point in points:
            current_count = 0
            current_layer = self.tree[current_count]
            while current_layer[3] != self.LEAF:
                if point[int(current_layer[0])] > current_layer[1]:
                    current_count += int(current_layer[3])
                    current_layer = self.tree[current_count]
                else:
                    current_count += int(current_layer[2])
                    current_layer = self.tree[current_count]
            if current_layer[3] == self.LEAF:
                ans = np.append(ans, current_layer[1])
        return ans

In [57]:
n = 100
x = np.random.uniform(0, 1, (n, 3))
y = np.zeros(n)

for i in range(n):
    y[i] = x[i][0] + x[i][1] + x[i][2]

y = y.reshape((n, 1))

data = np.concatenate((x, y), axis=1)

In [58]:
train_data = data[:60,]
test_data = data[60:,]

In [59]:
dtlearner = DecisionTreeLearner()

In [60]:
dtlearner.train(train_data)

In [61]:
dtlearner.query(test_data[:,:3])

array([1.97673023, 2.04405817, 1.97673023, 1.67541655, 2.32428113,
       1.7590017 , 2.32428113, 1.10154374, 1.42253617, 0.74303655,
       2.1533684 , 0.87759133, 1.62780181, 2.12936319, 2.32428113,
       0.96632162, 1.32108198, 1.97673023, 1.42874705, 0.653301  ,
       1.42253617, 1.66319226, 1.49929512, 1.4305536 , 1.80145162,
       0.9194574 , 1.97673023, 1.67541655, 0.653301  , 1.29239472,
       0.56148678, 1.66319226, 0.9194574 , 2.1533684 , 1.49929512,
       1.89372713, 1.64858085, 1.64858085, 2.34532171, 0.87759133])